## Deliverable 2. Create a Customer Travel Destinations Map.

In [54]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps
import numpy as np

# Import API keys from config.py file (one level up)
import sys
sys.path.insert(0, '../')  # Path goes here

from config import weather_api_key, g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [21]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("../Weather_Database/WeatherPy_database.csv")
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Description,Date
0,0,Punta Arenas,CL,-53.1500,-70.9167,42.91,87,100,4.61,overcast clouds,2022-05-20 20:54:10
1,1,Pevek,RU,69.7008,170.3133,32.92,97,79,5.48,broken clouds,2022-05-20 20:54:11
2,2,Altar,MX,30.7000,-111.8167,91.36,16,0,25.86,clear sky,2022-05-20 20:54:11
3,3,Jamestown,US,42.0970,-79.2353,76.95,78,100,8.99,overcast clouds,2022-05-20 20:54:12
4,4,Fortuna,US,40.5982,-124.1573,66.85,63,20,20.71,few clouds,2022-05-20 20:54:12


In [68]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 75
What is the maximum temperature you would like for your trip? 95


In [69]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Description,Date
2,2,Altar,MX,30.7000,-111.8167,91.36,16,0,25.86,clear sky,2022-05-20 20:54:11
3,3,Jamestown,US,42.0970,-79.2353,76.95,78,100,8.99,overcast clouds,2022-05-20 20:54:12
5,5,Hithadhoo,MV,-0.6000,73.0833,82.96,73,65,9.22,broken clouds,2022-05-20 20:54:13
9,9,Alofi,NU,-19.0595,-169.9187,76.89,94,40,5.75,scattered clouds,2022-05-20 20:54:15
10,10,Albany,US,42.6001,-73.9662,78.24,59,98,3.00,overcast clouds,2022-05-20 20:50:27
14,14,Bara,NG,10.3744,10.7288,86.50,47,54,8.05,broken clouds,2022-05-20 20:54:17
20,20,Ormara,PK,25.2088,64.6357,81.52,93,4,15.39,clear sky,2022-05-20 20:54:20
23,23,Saint George,US,37.1041,-113.5841,75.25,12,0,8.99,clear sky,2022-05-20 20:52:15
24,24,Matara,LK,5.9485,80.5353,76.87,93,46,7.90,scattered clouds,2022-05-20 20:54:21
33,33,Puliyur,IN,10.6333,78.8333,82.20,69,40,6.91,haze,2022-05-20 20:54:25


In [70]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.isnull().sum()

City_ID        0
City           0
Country        0
Lat            0
Lng            0
Max Temp       0
Humidity       0
Cloudiness     0
Wind Speed     0
Description    0
Date           0
dtype: int64

In [25]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.


In [71]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = preferred_cities_df[["City", "Country", "Max Temp", "Description", "Lat", "Lng"]].copy()


# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)



,City,Country,Max Temp,Description,Lat,Lng,Hotel Name
2,Altar,MX,91.36,clear sky,30.7000,-111.8167,
3,Jamestown,US,76.95,overcast clouds,42.0970,-79.2353,
5,Hithadhoo,MV,82.96,broken clouds,-0.6000,73.0833,
9,Alofi,NU,76.89,scattered clouds,-19.0595,-169.9187,
10,Albany,US,78.24,overcast clouds,42.6001,-73.9662,
14,Bara,NG,86.50,broken clouds,10.3744,10.7288,
20,Ormara,PK,81.52,clear sky,25.2088,64.6357,
23,Saint George,US,75.25,clear sky,37.1041,-113.5841,
24,Matara,LK,76.87,scattered clouds,5.9485,80.5353,
33,Puliyur,IN,82.20,haze,10.6333,78.8333,


In [72]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    params["location"] = f"{lat},{lng}"
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("hotel not found... skipping.")
        

hotel not found... skipping.
hotel not found... skipping.
hotel not found... skipping.
hotel not found... skipping.
hotel not found... skipping.
hotel not found... skipping.
hotel not found... skipping.
hotel not found... skipping.
hotel not found... skipping.
hotel not found... skipping.
hotel not found... skipping.
hotel not found... skipping.
hotel not found... skipping.
hotel not found... skipping.
hotel not found... skipping.
hotel not found... skipping.
hotel not found... skipping.
hotel not found... skipping.
hotel not found... skipping.
hotel not found... skipping.
hotel not found... skipping.


In [73]:
# 7. Drop the rows where there is no Hotel Name.
hotel_df['Hotel Name'].replace('', np.nan, inplace=True)
hotel_df.head(25)
hotel_df.dropna(subset=['Hotel Name'], inplace=True)
hotel_df.head(25)
hotel_df.isnull().sum()

City           0
Country        0
Max Temp       0
Description    0
Lat            0
Lng            0
Hotel Name     0
dtype: int64

In [74]:
# 8a. Create the output File (CSV)
output_data_file = "WeatherPy_vacation.csv"
# 8b. Export the City_Data into a csv
hotel_df.to_csv(output_data_file, index_label="City_ID")

In [75]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
<dt>Description</dt><dd>{Description}</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = hotel_df[["Lat", "Lng"]]

In [76]:
# 11a. Add a marker layer for each city to the map. 
# locations = hotel_df[["Lat", "Lng"]]
# max_temp = hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
# heat_layer = gmaps.heatmap_layer(locations, weights=max_temp,dissipating=False,
#              max_intensity=300, point_radius=4)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
# fig.add_layer(heat_layer)
fig.add_layer(marker_layer)
# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))